In [12]:
import sqlite3
import numpy as np
import matplotlib.pyplot as plt

In [13]:
db = '../example_data/bildverband2/datenbank.db'

In [14]:
from os import remove
remove(db)

In [15]:
from metadaten import metadaten
metadaten(db, '../example_data/bildverband2/*.JPG')

Metadaten
Bild
Bild
Bild
Bild
Bild
Bild


In [16]:
from find_aruco import find_aruco
find_aruco(db)

In [18]:
from find_sift import find_sift
find_sift(db)

../example_data/bildverband2/DJI_0289.JPG
../example_data/bildverband2/DJI_0288.JPG
../example_data/bildverband2/DJI_0287.JPG
../example_data/bildverband2/DJI_0286.JPG
../example_data/bildverband2/DJI_0285.JPG
../example_data/bildverband2/DJI_0284.JPG


In [19]:
from naeherungswerte import naeherungswerte
naeherungswerte(db)

Computed essential matrix: [[-2.84735068e-02 -1.00000000e+00  1.39196206e+03]
 [ 9.60918422e-01 -1.91129888e-02 -2.95142936e+03]
 [-1.05075132e+03  4.15872012e+03 -2.16741057e+06]]
[[0]
 [0]
 [0]
 ...
 [0]
 [0]
 [0]]
[[ 8.84628651e-01  2.59016000e-01 -3.87740714e-01  9.21677896e-01]
 [ 3.71065339e-01  1.12539938e-01  9.21762050e-01  3.87955882e-01]
 [ 2.82387435e-01 -9.59294258e-01  3.44419367e-03  2.97565067e-04]]


qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [20]:
from bundle_adjustment import  bundle_adjustment
bundle_adjustment(db)

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         1.7621e+11                                    3.02e+16    
       1              5         1.2342e+11      5.28e+10       1.92e+03       2.71e+16    
       2              6         8.2413e+10      4.10e+10       1.04e+03       8.40e+15    
       3              7         6.3122e+10      1.93e+10       7.80e+02       3.02e+15    
       4              8         5.7751e+10      5.37e+09       7.64e+02       4.72e+15    
       5              9         4.1696e+10      1.61e+10       5.18e+02       1.02e+15    
       6             10         3.3726e+10      7.97e+09       3.05e+02       1.67e+15    
       7             11         2.4834e+10      8.89e+09       2.61e+02       2.04e+14    
       8             13         2.2705e+10      2.13e+09       5.79e+01       1.18e+14    
       9             15         2.1896e+10      8.09e+08       5.89e+01       8.56e+13    

In [21]:
from join_nextpictures import join_nextpictures
join_nextpictures(db)


[[-1.08472678]
 [ 0.30711129]
 [ 0.3865093 ]] [[113.05918718]
 [ -6.162159  ]
 [  1.23318117]]


In [22]:
from join_nextcoords import join_nextcoords
join_nextcoords(db)

3609


qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [53]:
con = sqlite3.connect(db)
cur = con.cursor()

In [54]:
cur.execute("""SELECT lx,ly,lz FROM passpunkte WHERE lx is not null""")
coord = np.array(cur.fetchall())
cur.execute("""SELECT lx,ly,lz FROM bilder WHERE lx is not null""")
cams = np.array(cur.fetchall())

%matplotlib qt
fig = plt.figure()
fig.suptitle('3D reconstructed', fontsize=16)
ax = fig.add_subplot(projection='3d')
ax.plot(coord[:, 0], coord[:, 1], coord[:, 2], 'r.')
ax.plot(-cams[:, 0], -cams[:, 1], -cams[:, 2], 'b.')
ax.set_xlabel('x axis')
ax.set_ylabel('y axis')
ax.set_zlabel('z axis')
ax.view_init(elev=135, azim=90)
plt.axis('square')
ax.set_ylim([-2, 3])
ax.set_xlim([-2, 3])
ax.set_zlim([-2, 3])
plt.show()


qt.qpa.wayland: Wayland does not support QWindow::requestActivate()


In [47]:
cur.execute(
    """UPDATE kameras SET fx = 3000., fy = 3000., x0 = 2000., y0 = 1500.""")
cur.execute("""UPDATE bilder SET lx = NULL, ly = NULL, lz = NULL, lrx = NULL, lry = NULL, lrz = NULL""")
cur.execute(
    """UPDATE passpunkte SET lx = NULL, ly = NULL, lz = NULL""")
cur.execute("""UPDATE passpunktpos SET x = x*4000. ,y = y*4000.""")
con.commit()

In [50]:
cur.execute(
    """UPDATE bilder SET lx = NULL, ly = NULL, lz = NULL, lrx = NULL, lry = NULL, lrz = NULL""")
cur.execute(
    """UPDATE passpunkte SET lx = NULL, ly = NULL, lz = NULL""")
cur.execute("""UPDATE passpunktpos SET x = x/1000. ,y = y/1000.""")
con.commit()


In [40]:
fx = 3000
fy = 3000
x0 = 2000
y0 = 1500

K = np.array([[fx, 0, x0],
              [0, fy, y0],
              [0, 0, 1]])

t = np.array([2000,2000,1])

In [33]:
np.linalg.inv(K)@t


array([0.        , 0.16666667, 1.        ])

In [41]:
K = K / 4000.
K[2,2]=1
K

array([[0.75 , 0.   , 0.5  ],
       [0.   , 0.75 , 0.375],
       [0.   , 0.   , 1.   ]])

In [42]:
t =  np.array([0.5,0.5,1])

np.linalg.inv(K)@t


array([0.        , 0.16666667, 1.        ])